In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals

In [2]:
import json
import pandas as pd
import numpy as np
import pattern
import re
import json
from nltk.corpus import stopwords

# Détermination non supervisé des thématiques

## Préparer les données

In [3]:
DataBase=pd.read_csv("CleanedData/Tweets_DataBase.csv", sep='\t', index_col =0)

In [4]:
supp_stopWords=stopwords.words('french')+["galaxy_sssssg","egbw","eo_egbw","galaxy_s","plus","edge","_edge","merci","salut","coucou","concour","coup","mdrrr",'mdrrrr',"concours","mere","sssssg","gars","phone","maman","gold","smartphone","espere","plutot","pere","etre","iphone","concours","jojol","porno","samsung","telephone","merde","salut","mdrr"]+list(set(DataBase["Brand"]))[1:]+list(set(DataBase["Product"]))

In [5]:
D_Accents={"\xea":"e","\xe9":"e","\xe8":"e","\xe0":"a","\xfb":"u","\xf1":"n","\xf2":"o","\xf3":"o","\xf4":"o","\xf5":"o","\xf6":"o"}
for i in D_Accents.keys():
    supp_stopWords=[re.sub(i,D_Accents[i],x) for x in supp_stopWords]
DataBase['comment_relevant_Axis']=DataBase['comment_relevant_Axis'].apply(lambda x:  str(x).split())
DataBase['comment_relevant_Axis']=DataBase['comment_relevant_Axis'].apply(lambda x:[t for t in x if t not in supp_stopWords])

In [6]:
DataBase

,comment,Product,Brand,Date,comment_tokens,comment_lemmas,comment_relevant_Axis
0,C beau d avoir un smartphone denier cri mais p...,gen_smartphone,NaN,2016-12-26,"[[C, JJ, c], [beau, RB, beau], [d, JJ, d], [av...","[smartphone, denier, cri, notif, tre, tre]","[denier, notif]"
1,alxbrd bloquee sur smartphone gt droits d...,gen_smartphone,NaN,2016-12-26,"[[alxbrd, NN, alxbrd], [bloquee, NN, bloquee],...","[alxbrd, bloquee, smartphone, gt, droit, auteu...","[alxbrd, bloquee, droit, auteur, ordi]"
2,aucun Melle un bon Wiko suffit Ras le bol d...,gen_smartphone,NaN,2016-12-26,"[[aucun, DT, aucun], [Melle, NNP, melle], [un,...","[melle, wiko, ras, bol, societe, gen, smartphone]","[melle, wiko, societe]"
3,moi je veux gagner un smartphone je me suis c...,gen_smartphone,NaN,2016-12-26,"[[moi, PRP, moi], [je, PRP, je], [veux, VB, vo...","[smartphone, connecte, conte]","[connecte, conte]"
4,je sais que c est une ville tres culturel mai...,gen_smartphone,NaN,2016-12-26,"[[je, PRP, je], [sais, VB, savoir], [que, IN, ...","[ville, tre, dieu, merci, statues, smartphone]","[ville, dieu, statues]"
5,de la bombe a ce qu il para t and...,gen_smartphone,NaN,2016-12-26,"[[de, IN, de], [la, DT, la], [bombe, NN, bombe...","[bombe, qu, t, android, smartphone, galaxynote]","[bombe, android, galaxynote]"
6,J avais demande un etui de protection pour mon...,gen_smartphone,NaN,2016-12-26,"[[J, NNP, j], [avais, VB, avoir], [demande, VB...","[j, etui, protection, nveau, pere, l, dec, ecr...","[etui, protection, nveau, ecran, casse]"
7,ah mais j ai mal vu jai cru qu il tenait un v...,gen_smartphone,NaN,2016-12-26,"[[ah, UH, ah], [mais, CC, mais], [j, NN, j], [...","[j, jai, smartphone]",[]
8,a se monte le boule derriere son smartphone m...,gen_smartphone,NaN,2016-12-26,"[[a, VB, avoir], [se, PRP, se], [monte, VB, mo...","[boule, smartphone, louche, face]","[boule, louche, face]"
9,TopKDOTopAchat j ai besoin d un telephooooo...,gen_smartphone,NaN,2016-12-26,"[[TopKDOTopAchat, NN, topkdotopachat], [j, NN,...","[topkdotopachat, j, besoin, telephooooooneee, ...","[topkdotopachat, besoin, telephooooooneee, hon..."


## Filtrer les données

Garder uniquement les commentaires (génériques et particuliers) par rapport aux smatphones.

In [7]:
products=['gen_smartphone','iphone7plus','galaxy','iphone','galaxy s6','galaxy s7']
DataBase['test']=DataBase['Product'].apply(lambda x: x.lower() in products)
DataBase_smartphone=DataBase.loc[DataBase['test'] == True]
DataBase_smartphone['comment_relevant_Axis']

0                                          [denier, notif]
1                   [alxbrd, bloquee, droit, auteur, ordi]
2                                   [melle, wiko, societe]
3                                        [connecte, conte]
4                                   [ville, dieu, statues]
5                             [bombe, android, galaxynote]
6                  [etui, protection, nveau, ecran, casse]
7                                                       []
8                                    [boule, louche, face]
9        [topkdotopachat, besoin, telephooooooneee, hon...
10                                 [dieu, epoque, tronche]
11                                                      []
12                                [autonomie, utilisation]
13                                    [bigo, seum, snaper]
14                          [ecran, marque, pomme, niveau]
15                                        [android, roote]
16                    [forfait, nrjmobile, internet, wif

## Latent Semantic Analysis

### 1. Vectrorisation des données

In [8]:
corpus=DataBase_smartphone['comment_relevant_Axis'].apply(lambda x: " ".join(x))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1, stop_words = supp_stopWords)
X = vectorizer.fit_transform(corpus)

# les termes sont donnée par: 
vectorizer.get_feature_names()

[u'aavait',
 u'abbonez',
 u'abon',
 u'abonne',
 u'abord',
 u'absence',
 u'absorpt',
 u'absurdite',
 u'acce',
 u'accede',
 u'acceder',
 u'accent',
 u'accessoire',
 u'accident',
 u'accord',
 u'accueil',
 u'achat',
 u'achete',
 u'achetee',
 u'achetez',
 u'ackerman',
 u'actes',
 u'action',
 u'actions',
 u'activee',
 u'adaptateur',
 u'adaptation',
 u'adaptee',
 u'adida',
 u'adieu',
 u'adorai',
 u'adresse',
 u'aeration',
 u'aeroport',
 u'affaire',
 u'affaires',
 u'affichage',
 u'after',
 u'agacce',
 u'agent',
 u'agggghhhhh',
 u'aggression',
 u'ahah',
 u'ahahah',
 u'ahhh',
 u'ahhhh',
 u'ahre',
 u'aide',
 u'aipower',
 u'airframe',
 u'airpod',
 u'aise',
 u'alala',
 u'alan',
 u'alawadhy',
 u'album',
 u'alcatel',
 u'aleatoire',
 u'alentour',
 u'alessia',
 u'alexis',
 u'alice',
 u'alien',
 u'aliexpress',
 u'alizee',
 u'allah',
 u'alle',
 u'alleluia',
 u'aller',
 u'allocab',
 u'allocabpari',
 u'allume',
 u'almanzoq',
 u'alooor',
 u'alorrr',
 u'alors',
 u'aluminium',
 u'alxbrd',
 u'amandine',
 u'ama

### 2. Création du Bag-Of-Words (TF-iDF)

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)
tfidf

<4585x3204 sparse matrix of type '<type 'numpy.float64'>'
	with 11509 stored elements in Compressed Sparse Row format>

### 3. Décompsition en Valeur Singulière

##### La decomposition en valeurs singulière: M=USV
U:  est une matrice reelle orthogonale, (appartenant à l'éspace des commentaires)

S: matrice rectangulaire diagonale, avec les valeurs singulières en diagonale ordonnée du plus grande au plus petite

V: real or complex unitary matrix, (appartenant à l'éspace des )

La decomposition effectue un seuillage sur les valeurs singulières et ne garde que les k plus grandes valeurs singulière dans S (ainsi que les colonnes ou les lignes correspndantes aux valeurs gardées dans U et V) 

In [11]:
from scipy.sparse import linalg

K=60 # nombre de axes à garder
u, s0, v = linalg.svds(tfidf, k = K)
SV=np.zeros((K,K))
np.fill_diagonal(SV,s0)
VT=np.dot(np.transpose(v),SV)

### 4. Clustering K-means

In [12]:
from sklearn.cluster import KMeans
N_clust=10
KM_modele=KMeans(n_clusters=N_clust)
KM_modele.fit(VT)

import collections
clustering = collections.defaultdict(list)

for idx, label in enumerate(KM_modele.labels_):
    clustering[label].append(idx)

d ={}
for label in clustering :
    d[label] = []
    for i in clustering[label]:
        d[label] = d[label]+[vectorizer.get_feature_names()[i]]

In [13]:
for k in range(0,len(d)):
    print '---------'+'cluster '+ str(k)
    l=clustering[k][0:10]
    for i in l:
        #print(i)
        print(vectorizer.get_feature_names()[i])

---------cluster 0
aavait
abbonez
abon
abonne
abord
absence
absorpt
absurdite
acce
accede
---------cluster 1
photo
---------cluster 2
batterie
---------cluster 3
ecouteur
---------cluster 4
achete
---------cluster 5
temps
---------cluster 6
android
---------cluster 7
auriculaire
auriculaire_stereo
blanc
ble_blanc
ecouteurs
ecouteurs_intra
intra
soldes
stereo
---------cluster 8
chargeur
---------cluster 9
ecran


## Latent Dirichlet Allocation

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

### 1. Vectorizer les données

In [15]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 20)

tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
dtm_tf = tf_vectorizer.fit_transform(corpus)

### 2. Entrainement de l'algorithme de LDA

In [16]:
lda_tf = LatentDirichletAllocation(n_topics=12, random_state=0)
lda_tf.fit(dtm_tf)
lda_tfidf = LatentDirichletAllocation(n_topics=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=10, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

### 3. Visualisation des résultats

In [17]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      13.337547        1       1 -0.083388 -0.440053
10     10.810019        1       2 -0.074506  0.126915
1      10.531988        1       3 -0.059515  0.085568
0       8.742042        1       4 -0.060614  0.065927
4       8.473271        1       5 -0.065803  0.051538
3       8.396477        1       6  0.317690 -0.007637
5       7.696333        1       7 -0.063760  0.038105
2       7.683706        1       8  0.320451 -0.014491
8       6.782802        1       9 -0.056331  0.038239
7       6.033284        1      10 -0.065785  0.006246
9       6.009988        1      11 -0.055985  0.025713
11      5.502543        1      12 -0.052454  0.023929, topic_info=     Category        Freq         Term       Total  loglift  logprob
term                                                                
54    Default  161.000000        photo  161.000000  30.0000  30.0000
1     Default  128.000000      android  128.000000  29.0000  29.0000
39    Default  112.000000         jour  112.000000  28.0000  28.0000
77    Default   94.000000        video   94.000000  27.0000  27.0000
27    Default  103.000000        ecran  103.000000  26.0000  26.0000
24    Default   79.000000         deja   79.000000  25.0000  25.0000
49    Default   65.000000         mort   65.000000  24.0000  24.0000
8     Default   90.000000     batterie   90.000000  23.0000  23.0000
19    Default   90.000000     chargeur   90.000000  22.0000  22.0000
0     Default   65.000000       achete   65.000000  21.0000  21.0000
22    Default   77.000000        coque   77.000000  20.0000  20.0000
47    Default   63.000000         mois   63.000000  19.0000  19.0000
59    Default   58.000000      qualite   58.000000  18.0000  18.0000
42    Default   48.000000         jvai   48.000000  17.0000  17.0000
25    Default   71.000000     ecouteur   71.000000  16.0000  16.0000
74    Default   48.000000      twitter   48.000000  15.0000  15.0000
66    Default   58.000000       soldes   58.000000  14.0000  14.0000
28    Default   43.000000        envie   43.000000  13.0000  13.0000
29    Default   52.000000         fois   52.000000  12.0000  12.0000
69    Default   48.000000        temps   48.000000  11.0000  11.0000
57    Default   43.000000     probleme   43.000000  10.0000  10.0000
65    Default   40.000000         snap   40.000000   9.0000   9.0000
3     Default   43.000000        appli   43.000000   8.0000   8.0000
15    Default   43.000000        casse   43.000000   7.0000   7.0000
33    Default   38.000000        genre   38.000000   6.0000   6.0000
50    Default   31.000000      musique   31.000000   5.0000   5.0000
52    Default   31.000000        offre   31.000000   4.0000   4.0000
23    Default   30.000000   correcteur   30.000000   3.0000   3.0000
51    Default   43.000000       nougat   43.000000   2.0000   2.0000
48    Default   33.000000        monde   33.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
78    Topic12    0.106089         wifi   24.542799  -2.5439  -7.4425
46    Topic12    0.106090         mode   24.820589  -2.5552  -7.4425
14    Topic12    0.106089       camera   24.823068  -2.5553  -7.4425
79    Topic12    0.106090         wiko   25.281260  -2.5736  -7.4425
63    Topic12    0.106089         reve   25.487272  -2.5817  -7.4425
35    Topic12    0.106089        heure   25.624126  -2.5871  -7.4425
71    Topic12    0.106090       trempe   25.888524  -2.5973  -7.4425
61    Topic12    0.106089     question   25.989780  -2.6012  -7.4425
11    Topic12    0.106089        blanc   26.027989  -2.6027  -7.4425
36    Topic12    0.106089        intra   26.027989  -2.6027  -7.4425
7     Topic12    0.106089  auriculaire   26.027989  -2.6027  -7.4425
26    Topic12    0.106088    ecouteurs   26.027986  -2.6027  -7.4425
51    Topic12    0.106632       nougat   43.929865  -3.1210  -7.4374
5     Topic12    0.106109  

In [18]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      12.181086        1       1  0.152246  0.406062
2      11.999303        1       2 -0.265038  0.014910
6      11.781654        1       3  0.131997 -0.159994
8      11.668906        1       4  0.134690 -0.163617
7      10.580003        1       5 -0.193007  0.017444
3       8.978500        1       6  0.077336 -0.036123
5       8.795984        1       7  0.089002 -0.035141
9       8.374175        1       8 -0.234383  0.010739
0       7.846275        1       9  0.053118 -0.027024
1       7.794114        1      10  0.054039 -0.027257, topic_info=     Category        Freq        Term       Total  loglift  logprob
term                                                               
54    Default  112.000000       photo  112.000000  30.0000  30.0000
8     Default   91.000000    batterie   91.000000  29.0000  29.0000
1     Default   84.000000     android   84.000000  28.0000  28.0000
19    Default   83.000000    chargeur   83.000000  27.0000  27.0000
39    Default   75.000000        jour   75.000000  26.0000  26.0000
27    Default   74.000000       ecran   74.000000  25.0000  25.0000
77    Default   64.000000       video   64.000000  24.0000  24.0000
25    Default   64.000000    ecouteur   64.000000  23.0000  23.0000
22    Default   56.000000       coque   56.000000  22.0000  22.0000
0     Default   47.000000      achete   47.000000  21.0000  21.0000
49    Default   47.000000        mort   47.000000  20.0000  20.0000
60    Default   45.000000      quelqu   45.000000  19.0000  19.0000
24    Default   50.000000        deja   50.000000  18.0000  18.0000
3     Default   39.000000       appli   39.000000  17.0000  17.0000
74    Default   39.000000     twitter   39.000000  16.0000  16.0000
47    Default   42.000000        mois   42.000000  15.0000  15.0000
65    Default   36.000000        snap   36.000000  14.0000  14.0000
2     Default   33.000000       annee   33.000000  13.0000  13.0000
72    Default   33.000000        truc   33.000000  12.0000  12.0000
59    Default   39.000000     qualite   39.000000  11.0000  11.0000
69    Default   37.000000       temps   37.000000  10.0000  10.0000
28    Default   33.000000       envie   33.000000   9.0000   9.0000
29    Default   36.000000        fois   36.000000   8.0000   8.0000
44    Default   28.000000      marche   28.000000   7.0000   7.0000
43    Default   28.000000        main   28.000000   6.0000   6.0000
40    Default   27.000000     journee   27.000000   5.0000   5.0000
5     Default   33.000000        apre   33.000000   4.0000   4.0000
48    Default   28.000000       monde   28.000000   3.0000   3.0000
32    Default   27.000000       frere   27.000000   2.0000   2.0000
30    Default   26.000000        fond   26.000000   1.0000   1.0000
...       ...         ...         ...         ...      ...      ...
35    Topic10    0.122459       heure   17.794978  -2.4271  -7.3906
18    Topic10    0.122459      charge   18.006337  -2.4389  -7.3906
46    Topic10    0.122459        mode   18.084935  -2.4433  -7.3906
23    Topic10    0.122459  correcteur   18.181134  -2.4486  -7.3906
55    Topic10    0.122459        pote   18.262357  -2.4530  -7.3906
70    Topic10    0.122459       train   18.418567  -2.4615  -7.3906
10    Topic10    0.122459     bientot   18.675648  -2.4754  -7.3906
31    Topic10    0.122459     forfait   20.392457  -2.5633  -7.3906
17    Topic10    0.122461      change   20.499763  -2.5686  -7.3906
6     Topic10    0.122459     aujourd   20.611651  -2.5740  -7.3906
50    Topic10    0.122467     musique   20.694325  -2.5780  -7.3906
12    Topic10    0.122459        bout   21.198716  -2.6021  -7.3906
38    Topic10    0.122640   iphoneplu   21.938371  -2.6349  -7.3891
27    Topic10    0.122605       ecran   74.362354  -3.8559  -7.3894
29    Topic10    0.122500        fois   36.332519  -3.1406  -7.3903
1     Topic10    0.122492     android   84.57

# Enrichissement de la base: spécification des thématiques 

### Nous allons l'appliquer pour les thématiques passées en paramètre

Comme chaque thématique est caractérisée par un cluster des mots considérés synonymes (par rapport aux données), nous pensons à les déterminer en se basant sur la similarité défine par le modèle Word2Vec entrainé sur les données.

#### Ajouter les données de Wikipedia 

In [19]:
import requests
from bs4 import BeautifulSoup
import nltk.data
tokenizer = nltk.data.load("tokenizers/punkt/french.pickle")
from pattern.fr import parse, split

In [75]:
wikis=['https://fr.wikipedia.org/wiki/Samsung_Galaxy', 'https://fr.wikipedia.org/wiki/IPhone','https://fr.wikipedia.org/wiki/Smartphone','https://fr.wikipedia.org/wiki/T%C3%A9l%C3%A9phonie_mobile']

wikis_sentences=[]
for wiki in wikis:
    try:
        wiki_sentences= tokenizer.tokenize(BeautifulSoup(requests.get(wiki).text,'html.parser').get_text())
        wikis_sentences+=wiki_sentences
    except:
        continue

Wiki=pd.DataFrame(wikis_sentences)
Wiki

""


In [483]:
def NounsOnly(x):
    y=[]
    for t in x:
        if len(t)==3:
            if t[1] in ['NN','NNS','NNPS','NNP'] :
                y =+t[2]
    return y 

In [524]:
D_Accents={"\\xea":"e","\\xe9":"e","\\xe8":"e","\\xe0":"a","\\xfb":"u","\\xf1":"n","\\xf2":"o","\\xf3":"o","\\xf4":"o","\\xf5":"o","\\xf6":"o"}
for i in D_Accents.keys():
    Wiki[0]=Wiki[0].apply(lambda x: re.sub(i,D_Accents[i],x))

Wiki[0]=Wiki[0].apply(lambda x: re.sub(r'[/:\(\)\%+0-9\n_]+', ' ', x))
Wiki[0]=Wiki[0].apply(lambda x: re.sub(r'[\[\]-]+', ' ', x))
Wiki[0]=Wiki[0].apply(lambda x: re.sub(r'\\([A-Z0-9a-z]+)', '', x))
Wiki[0]=Wiki[0].apply(lambda x: re.sub(r'[",]+', ' ', x))

In [525]:
Wiki["test"]=Wiki[0].apply(lambda x: len(x.split())>0)
DataBase_Wiki=Wiki.loc[Wiki['test'] == True]

In [526]:
DataBase_Wiki[1]=DataBase_Wiki[0].apply(lambda x: parse(x, chunks = False, lemmata = True).split()[0])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [527]:
DataBase_Wiki[2]=DataBase_Wiki[1].apply(lambda x: [t[2] for t in x if t[1] in ['NN','NNS','NNPS','NNP'] ])
DataBase_Wiki[2]=DataBase_Wiki[2].apply(lambda x: [t for t in x if len(t)>3])
DataBase_Wiki[2]=DataBase_Wiki[2].apply(lambda x: [t for t in x if len(t)<17])

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://panda

In [529]:
DataBase_Wiki[3]=DataBase_Wiki[2].apply(lambda x:[t for t in x if t not in supp_stopWords])
DataBase_Wiki[4]=DataBase_Wiki[3].apply(lambda x:  " ".join(x))

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


#### Les thématiques: paramétrage supervisé

In [39]:
thematiques={'1':["ecran"],'2':["chargeur","batterie"],'3':["prix"],'4':["couleur","coque"],'5':["photo","camera"],'6':['son','auriculaire_stereo','auriculaire','ecouteurs']}
def Thematiques(s):
    x=str(s)
    if x in thematiques.keys():
        return thematiques[x][0]
    return x

#### Les clusters des thématiques: relation de similarité fournie par Word2vec

In [40]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 15   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 7           # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec
print "Training model..."

Training model...


In [41]:
sentences= list(list(DataBase_smartphone['comment_relevant_Axis']))#+list(DataBase_Wiki[3]))

In [42]:
%time model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context)

CPU times: user 122 ms, sys: 32.3 ms, total: 155 ms
Wall time: 118 ms


In [43]:
model.vocab

{'achete': <gensim.models.word2vec.Vocab at 0x12011bb10>,
 'adresse': <gensim.models.word2vec.Vocab at 0x12011bb50>,
 'android': <gensim.models.word2vec.Vocab at 0x12011ba90>,
 'annee': <gensim.models.word2vec.Vocab at 0x120118850>,
 'appareil': <gensim.models.word2vec.Vocab at 0x120100ed0>,
 'appli': <gensim.models.word2vec.Vocab at 0x12011b410>,
 'application': <gensim.models.word2vec.Vocab at 0x12011b150>,
 'apre': <gensim.models.word2vec.Vocab at 0x120100d50>,
 'argent': <gensim.models.word2vec.Vocab at 0x120118650>,
 'aujourd': <gensim.models.word2vec.Vocab at 0x120118e50>,
 'aujourd_hui': <gensim.models.word2vec.Vocab at 0x120118d90>,
 'auriculaire': <gensim.models.word2vec.Vocab at 0x12011b190>,
 'auriculaire_stereo': <gensim.models.word2vec.Vocab at 0x12011b450>,
 'batterie': <gensim.models.word2vec.Vocab at 0x120100fd0>,
 'besoin': <gensim.models.word2vec.Vocab at 0x120118590>,
 'bientot': <gensim.models.word2vec.Vocab at 0x120100950>,
 'blague': <gensim.models.word2vec.Vocab 

In [44]:
model.most_similar("auriculaire")

[('stereo', 0.5570544004440308),
 ('intra', 0.5425379872322083),
 ('ble_blanc', 0.5292438268661499),
 ('ecouteurs_intra', 0.5284942984580994),
 ('blanc', 0.4991714358329773),
 ('auriculaire_stereo', 0.49069446325302124),
 ('ecouteurs', 0.48037588596343994),
 ('soldes', 0.4059433937072754),
 ('verre', 0.3220933675765991),
 ('ecran', 0.30474627017974854)]

In [47]:
import itertools
thematiques_clusters = {}
for theme in thematiques.keys() :
    thematiques_clusters[theme]=[]
    others=thematiques.copy()
    otherslist=list(itertools.chain.from_iterable( others.values()))
    #otherslist=[val in v for v in others.values()]
    del others[theme]
    for mot in thematiques[theme]:
        thematiques_clusters[theme]+= [mot]
        try:
            thematiques_clusters[theme] += [t[0] for t in model.most_similar(mot)[:3] if( t[1] > 0.54 and t[0] not in otherslist)]
        except:
            print(mot +" : ce mot n'est pas dans le vocabulaire")
    thematiques_clusters[theme]=list(set(thematiques_clusters[theme]))
        #thematiques_clusters[theme] = [t[0] for t in model.most_similar(positive=thematiques[theme])]+thematiques[theme]

son : ce mot n'est pas dans le vocabulaire


In [48]:
thematiques_clusters

{u'1': [u'ecran'],
 u'2': [u'batterie', u'chargeur'],
 u'3': [u'prix'],
 u'4': [u'couleur', u'coque'],
 u'5': [u'photo', u'camera'],
 u'6': ['stereo',
  u'ecouteurs',
  u'auriculaire_stereo',
  u'son',
  u'auriculaire',
  'intra']}

In [49]:
def ThemeCluster(s):
    l=[]
    for theme in thematiques_clusters.keys():
        if (any([t in s for t in thematiques_clusters[theme]])):
            l+=[theme]
    if l==[]:
        return ["General"]
    return l

In [50]:
DataBase_smartphone['Thematiques_code']=DataBase_smartphone['comment_relevant_Axis'].apply(ThemeCluster)
rows = []
_ = DataBase_smartphone.apply(lambda row: [rows.append([row['comment'], row['Product'], row['Brand'],row['Date'], nn]) 
                         for nn in row.Thematiques_code], axis=1)
DataBase_smartphone_new = pd.DataFrame(rows, columns=['comment','Product','Brande','Date','Thematiques_code'])
DataBase_smartphone_new['Thematiques']=DataBase_smartphone_new['Thematiques_code'].apply(Thematiques)

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [51]:
DataBase_smartphone_new

,comment,Product,Brande,Date,Thematiques_code,Thematiques
0,C beau d avoir un smartphone denier cri mais p...,gen_smartphone,NaN,2016-12-26,General,General
1,alxbrd bloquee sur smartphone gt droits d...,gen_smartphone,NaN,2016-12-26,General,General
2,aucun Melle un bon Wiko suffit Ras le bol d...,gen_smartphone,NaN,2016-12-26,General,General
3,moi je veux gagner un smartphone je me suis c...,gen_smartphone,NaN,2016-12-26,General,General
4,je sais que c est une ville tres culturel mai...,gen_smartphone,NaN,2016-12-26,General,General
5,de la bombe a ce qu il para t and...,gen_smartphone,NaN,2016-12-26,General,General
6,J avais demande un etui de protection pour mon...,gen_smartphone,NaN,2016-12-26,1,ecran
7,ah mais j ai mal vu jai cru qu il tenait un v...,gen_smartphone,NaN,2016-12-26,General,General
8,a se monte le boule derriere son smartphone m...,gen_smartphone,NaN,2016-12-26,General,General
9,TopKDOTopAchat j ai besoin d un telephooooo...,gen_smartphone,NaN,2016-12-26,General,General


## LOAD

In [52]:
DataBase_smartphone_new.to_csv("CleanedData/DataBase_Axis.csv", sep=str(","))